# self_intro_character_identification
Since we've explored the visual, audio, and subtitle tracks and extracted features from each, we can start to use them all together to accomplish broader *Moviegoer* goals. This notebook is the first example of this.

We'll be generating a list of possible characters, then looking for these names in self-identifications ("My name is Alice." or "I'm Ben."). Then we can build a composite, "average" encoding of their face, so we can track them throughout the film, every time we spot their face.

In [1]:
import sys
sys.path.append('../subtitle_features')
from subtitle_dataframes_io import *
from subtitle_auxiliary_io import *
sys.path.append('../vision_features')
from vision_dataframes_io import *
sys.path.append('../audio_features')
from audio_dataframes_io import *
from time_reference_io import *
import datetime

pd.set_option('display.max_colwidth', None)
nlp = spacy.load('en')

## Generating the Film's Character List
We'll generate the `subtitle_df` and `sentence_df` dataframes based off the subtitle file.

In [2]:
subs = pysrt.open('../subtitles/plus_one.srt')
subtitle_df = generate_base_subtitle_df(subs)
subtitle_df = generate_subtitle_features(subtitle_df)
subtitle_df['cleaned_text'] = subtitle_df['concat_sep_text'].map(clean_line)
sentences = partition_sentences(remove_blanks(subtitle_df['cleaned_text'].tolist()), nlp)
subtitle_indices = tie_sentence_subtitle_indices(sentences, subtitle_df)
sentence_df = pd.DataFrame(list(zip(sentences, subtitle_indices)), columns=['sentence', 'subtitle_indices'])
sentence_df = generate_sentence_features(sentence_df, nlp)

We've previously defined two functions that will read through the subtitles and count up character names, either mentioned as dialogue, or labelling an offscreen speaker.

In [3]:
chars_sub_mentions = character_subtitle_mentions(sentences, nlp)
chars_sub_mentions

[('Ben', 95),
 ('Alice', 33),
 ('Dad', 21),
 ('Gina', 11),
 ('Brett', 7),
 ('Nick', 6),
 ('Matt', 5),
 ('Jess Ramsey', 5),
 ('Amanda', 4),
 ('Ben King', 4)]

In [4]:
chars_offscreen_speakers = character_offscreen_speakers(subtitle_df)
chars_offscreen_speakers

[('ALICE', 51),
 ('BEN', 47),
 ('CHUCK', 14),
 ('ANGELA', 5),
 ('NICK', 3),
 ('DAVIS', 3),
 ('MATT', 3),
 ('DEEJAY', 2),
 ('BRETT', 2),
 ('NATE', 1)]

We can take the most common names, and assume they're the main characters.

In [5]:
characters = []

for character in chars_sub_mentions:
    if character[1] >= 10:
        characters.append(character[0].lower())

for character in chars_offscreen_speakers:
    if character[1] >= 5: 
        characters.append(character[0].lower())
        
characters = list(set(characters))
characters

['angela', 'alice', 'dad', 'gina', 'chuck', 'ben']

## Finding Self-Introduction Sentences
For this exercise, we're going to identify characters solely based on self-introductions. We have a function to find phrases like "My name is Alice." or "I'm Ben."

In [6]:
sentence_df[sentence_df['self_intro'].notnull()]

,sentence,subtitle_indices,self_intro,other_intro,direct_address,conv_boundary
539,"I'm Ben, by the way.""",[506],Ben,None,None,None
564,I'm Ben.,[527],Ben,None,None,None
566,I'm Kara.,[528],Kara,None,None,None
748,"I'm Ellie, by the way.",[697],Ellie,None,None,None
752,I'm Alice.,[701],Alice,None,None,None
909,I'm Ben King.,[843],Ben King,None,None,None
2324,"Uh, I'm Ben.",[2068],Ben,None,None,None
2327,I'm Alice.,[2069],Alice,None,None,None
2331,I'm Jackie.,[2070],Jackie,None,None,None
2961,"I'm Ben, and most of you here probably know me as Chuck's son.","[2619, 2620]",Ben,None,None,None


For now, we'll focus on Ben's self-introductions. The film has five sentences where he introduces himself. 

In [7]:
ben_string = 'ben'

In [8]:
sentence_df[sentence_df.self_intro.str.contains(ben_string, na=False, case=False)]

,sentence,subtitle_indices,self_intro,other_intro,direct_address,conv_boundary
539,"I'm Ben, by the way.""",[506],Ben,None,None,None
564,I'm Ben.,[527],Ben,None,None,None
909,I'm Ben King.,[843],Ben King,None,None,None
2324,"Uh, I'm Ben.",[2068],Ben,None,None,None
2961,"I'm Ben, and most of you here probably know me as Chuck's son.","[2619, 2620]",Ben,None,None,None


## Calculating the Film Times
From the `sentence_df`, we need to look for Ben's self-introductions and find the indices in `subtitle_df`, which contains times of the actual subtitles (not sentences).

In [9]:
ben_indices = sentence_df[sentence_df.self_intro.str.contains(ben_string, na=False, case=False)].subtitle_indices.values
ben_indices

array([list([506]), list([527]), list([843]), list([2068]),
       list([2619, 2620])], dtype=object)

In [10]:
ben_flattened_indices = np.concatenate(ben_indices).ravel()
ben_flattened_indices

array([ 506,  527,  843, 2068, 2619, 2620])

We can see that one of these sentences spans two separate subtitles. We'll leave both of these in, because if it spans such a long duration, there's a good chance it'll have Ben's face onscreen.

In [11]:
subtitle_df[subtitle_df.index.isin(ben_flattened_indices)]

,srt_index,original_text,start_time,end_time,concat_sep_text,separated_flag,laugh,hesitation,speaker,music,parenthetical,el_parenthetical,el_italic,cleaned_text
506,450,"I'm Ben, by the way.""",00:17:57.244000,00:17:58.662000,"I'm Ben, by the way.""",0,0,0,None,0,None,0,0,"I'm Ben, by the way."""
527,466,- I'm Ben.\n- Hi. I'm Kara.,00:18:26.356000,00:18:28.066000,I'm Ben.,1,0,0,None,0,None,0,0,I'm Ben.
843,747,It's the same thing with me.\nI'm Ben King.,00:29:33.982000,00:29:35.650000,It's the same thing with me. I'm Ben King.,0,0,0,None,0,None,0,0,It's the same thing with me. I'm Ben King.
2068,1742,"Uh, I'm Ben.\nThis is Alice.",01:10:01.448000,01:10:03.200000,"Uh, I'm Ben. This is Alice.",0,0,0,None,0,None,0,0,"Uh, I'm Ben. This is Alice."
2619,2232,"I'm Ben,",01:31:08.547000,01:31:10.341000,"I'm Ben,",0,0,0,None,0,None,0,0,"I'm Ben,"
2620,2233,and most of you here\nprobably know me as Chuck's son.,01:31:10.383000,01:31:13.427000,and most of you here probably know me as Chuck's son.,0,0,0,None,0,None,0,0,and most of you here probably know me as Chuck's son.


For each subtitle, we'll calculate the `mid_time`, or the difference between the start and end times.

In [12]:
mid_time = subtitle_mid_time(subtitle_df.iloc[506].start_time, subtitle_df.iloc[506].end_time)
time_to_frame(mid_time)

1078

In [13]:
mid_time_frames = []
for sub_index in ben_flattened_indices:
    mid_time = subtitle_mid_time(subtitle_df.iloc[sub_index].start_time, subtitle_df.iloc[sub_index].end_time)
    mid_time_frames.append(time_to_frame(mid_time))

mid_time_frames

[1078, 1107, 1775, 4202, 5469, 5472]